In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import plotly.graph_objects as go
import numpy as np

# 定义 LSTM 降噪模型
class LSTMDenoiser(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

# 初始化模型
model = LSTMDenoiser(input_size=1, hidden_size=64, num_layers=2)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 导出模型为 ONNX 格式
input_data = torch.randn(32, 100, 1)
onnx_path = "./models/lstm_denoiser.onnx"
torch.onnx.export(
    model,
    input_data,
    onnx_path,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size", 1: "sequence_length"},
                  "output": {0: "batch_size", 1: "sequence_length"}},
    opset_version=11,
)
print(f"Model exported to {onnx_path}")

# 示例训练数据
batch_size = 32
sequence_length = 100
input_size = 1

torch.manual_seed(0)
noisy_signal = torch.randn(batch_size, sequence_length, input_size)  # 噪声信号
clean_signal = torch.sin(
    torch.linspace(0, 2 * np.pi, sequence_length)
    ).repeat(batch_size, 1).unsqueeze(-1)  # 干净信号

# 用于保存训练损失
losses = []

# 绘制原始噪声信号与干净信号的对比
fig1 = go.Figure()
fig1.add_trace(go.Scatter(y=noisy_signal[0].squeeze().numpy(), mode='lines', name='Noisy Signal'))
fig1.add_trace(go.Scatter(y=clean_signal[0].squeeze().numpy(), mode='lines', name='Clean Signal'))
fig1.update_layout(title='Original Signals', xaxis_title='Time Step', yaxis_title='Signal Value')
fig1.show()

# 模型训练
for epoch in range(10):
    optimizer.zero_grad()
    output = model(noisy_signal)
    loss = criterion(output, clean_signal)
    loss.backward()
    optimizer.step()

    losses.append(loss.item())
    print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")

# 绘制训练损失曲线
fig2 = go.Figure()
fig2.add_trace(go.Scatter(y=losses, mode='lines+markers', name='Training Loss'))
fig2.update_layout(title='Training Loss Curve', xaxis_title='Epoch', yaxis_title='Loss')
fig2.show()

# 预测降噪结果
with torch.no_grad():
    denoised_signal = model(noisy_signal)

# 绘制降噪结果与干净信号的对比
fig3 = go.Figure()
fig3.add_trace(go.Scatter(y=clean_signal[0].squeeze().numpy(), mode='lines', name='Clean Signal'))
fig3.add_trace(go.Scatter(y=denoised_signal[0].squeeze().numpy(), mode='lines', name='Denoised Signal'))
fig3.update_layout(title='Denoised vs Clean Signal', xaxis_title='Time Step', yaxis_title='Signal Value')
fig3.show()


d:\Foester\initial preparation\noise_reduction\.venv\Lib\site-packages\torch\onnx\symbolic_opset9.py:4279: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


Model exported to ./models/lstm_denoiser.onnx


Epoch 1, Loss: 0.5130
Epoch 2, Loss: 0.5066
Epoch 3, Loss: 0.5016
Epoch 4, Loss: 0.4979
Epoch 5, Loss: 0.4954
Epoch 6, Loss: 0.4942
Epoch 7, Loss: 0.4941
Epoch 8, Loss: 0.4948
Epoch 9, Loss: 0.4958
Epoch 10, Loss: 0.4964


In [2]:
# 验证onnx格式模型
import onnxruntime as ort
import numpy as np

# 加载 ONNX 模型
session = ort.InferenceSession("models/lstm_denoiser.onnx")

# 创建输入数据
input_data = np.random.randn(32, 100, 1).astype(np.float32)

# 推理
outputs = session.run(["output"], {"input": input_data})
print("Output shape:", np.array(outputs).shape)


Output shape: (1, 32, 100, 1)
